## Deployment

Deployment notes and scripts - local node and polygon test net

Prior to starting Jupyter be sure node js is avaiable - set it via nvm if needed.

NOTE: This copy of the book deploys two domains to the same test net. This is done to troubleshoot the set up prior to two chain deployment.

In [ ]:
import json

from web3 import Web3
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))

In [ ]:
# Eth test account addresses
ACCT0='0x892BB2e4F6b14a2B5b82Ba8d33E5925D42D4431F'
ACCT1='0x9949f7e672a568bB3EBEB777D5e8D1c1107e96E5'
ACCT2='0x835F0Aa692b8eBCdEa8E64742e5Bce30303669c2'
ACCT3='0x7bA7d161F9E8B707694f434d65c218a1F0853B1C'
ACCT4='0xB4C3D79CDC0eb7A8576a8bf224Bbc6Bec790c320'
ACCT5='0x5Ad35F89D8C1d03089BDe2578Ce43883E3f2A7B0'
ACCT6='0x0234643975F308b76d1241897e7d70b02C155daa'
ACCT7='0x5199524B11e801c52161CA76dB9BFD72f4a4E1E1'
ACCT8='0x549381D65fe61046911d11743D5c0941Ed704640'
ACCT9='0x73dA1eD554De26C467d97ADE090af6d52851745E'

In [ ]:
# Eth test account private keys
ACCT0KEY='0xcb1a18dff8cfcee16202bf86f1f89f8b3881107b8192cd06836fda9dbc0fde1b'
ACCT1KEY='0xa54f24f80839b659fc44fbb19492507bc734ea572f6f5672787cd8e9a198bf28'
ACCT2KEY='0x824f9e081f93102ca26e9e696bb6804079a5e4e3fca3a05216e2b6e0538fcab9'
ACCT3KEY='0x3e56e9b2db8389123a03816c37dd4515e07077483fbe865156a2d8f6003b6725'
ACCT4KEY='0xca55c5904f97405816f8d24c5e7810aaec2aa347ea0e1f920f19eca05669ae7b'
ACCT5KEY='0x6100006a16d6a0fd065f62165e64d412920afd0e8fc59956ad7a9116e363b72e'
ACCT6KEY='0x55b0e7919eb08e618dbddf025341094eefc0db60244736c1faac1412d68868f3'
ACCT7KEY='0x81b475051c4686cffe815e50216a9bac397c0e5d9108170823b290d41823fa8a'
ACCT8KEY='0xc3599ced1484dc8eccbb477cf8c318ca48ef1f2aac81a10a3774fe40d3b678de'
ACCT9KEY='0xf9832eeac47db42efeb2eca01e6479bfde00fda8fdd0624d45efd0e4b9ddcd3b'

In [ ]:
# Polygon config
import os

PG=os.environ['PG']
print(PG)
PGKEY='0x' + os.environ['PGKEY']
PG_URL=os.environ['PG_URL']
print(PG_URL)

In [ ]:
mbw3 = Web3(Web3.HTTPProvider(PG_URL))

In [ ]:
from web3.middleware import geth_poa_middleware
mbw3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [ ]:
domain_1_acct = ACCT0
domain_1_key = ACCT0KEY

domain_2_acct = PG
domain_2_key = PGKEY

domain_2_mint_recipient = ACCT4
domain_2_mint_recipient_key = ACCT4KEY

attestor = ACCT8
attestor_key = ACCT8KEY

In [ ]:
print(domain_1_acct)

## Fiddy Deployment - Domain 1

1. Pick account for deploying FIDDY
2. Deploy it
3. Grab the address









In [ ]:
home = ! echo $HOME
exec_home = home[0] + '/code/cctp-components'
print(exec_home)

In [ ]:
# Fiddy deployment
!cd {exec_home} && DEPLOYER_KEY={domain_1_key} OWNER_ADDRESS={domain_1_acct} forge script script/Token.s.sol:DeployScript --broadcast --rpc-url http://127.0.0.1:8545 --extra-output-files abi --extra-output-files bin


In [ ]:
%%bash -s "$exec_home" --out DOMAIN1_FIDDY_ADDRESS
cd $1
ls
node script/deploy-details.js

In [ ]:
DOMAIN1_FIDDY_ADDRESS = DOMAIN1_FIDDY_ADDRESS.strip()
print(DOMAIN1_FIDDY_ADDRESS)

## Fiddy Domain 2 Deployment

1. Pick account for deploying FIDDY
2. Deploy it
3. Grab the address

In [ ]:
# PG deployment
!cd {exec_home} && DEPLOYER_KEY={domain_2_key} OWNER_ADDRESS={domain_2_acct} forge script script/Token.s.sol:DeployScript --broadcast --rpc-url {PG_URL} --extra-output-files abi --extra-output-files bin


In [ ]:
%%bash -s "$exec_home" --out DOMAIN2_FIDDY_ADDRESS
cd $1
ls
node script/pgdeploy-details.js

In [ ]:
DOMAIN2_FIDDY_ADDRESS = DOMAIN2_FIDDY_ADDRESS.strip()
print(DOMAIN2_FIDDY_ADDRESS)

## CCTP Set Up - Domain 1


In [ ]:
# Adapted from https://github.com/circlefin/evm-cctp-contracts/blob/master/scripts/precomputeRemoteMessengerAddress.py

from web3 import Web3

import os
import rlp

def compute_address(sender, nonce):
    """
    Computes the address for a deployed contract given the address
    of the sender and the current nonce.
    """
    sender_as_bytes = Web3.toBytes(hexstr=sender)
    contract_address = Web3.toHex(Web3.keccak(rlp.encode([sender_as_bytes, nonce])))[-40:]
    return contract_address

def precompute_remote_token_messenger_address(remote_rpc_url, remote_deployer):
    """
    Computes expected address for remote token messenger contract on the
    input remote_rpc_url
    """
    remote_token_messenger_deployer = remote_deployer
    remote_domain_node = Web3(Web3.HTTPProvider(remote_rpc_url))

    remote_token_messenger_deployer_nonce =  remote_domain_node.eth.get_transaction_count(remote_token_messenger_deployer)
    
    remote_token_messenger_address = compute_address(remote_token_messenger_deployer, remote_token_messenger_deployer_nonce)

    return remote_token_messenger_address



In [ ]:
domain2TokenMessengerAddress = precompute_remote_token_messenger_address('http://localhost:8545', domain_2_acct)
print(domain2TokenMessengerAddress)

In [ ]:
home = ! echo $HOME
cctp_home = home[0] + '/code/evm-cctp-contracts'
print(cctp_home)

In [ ]:
%%bash -s "$cctp_home" "$domain_1_acct" "$domain_1_key" "$DOMAIN1_FIDDY_ADDRESS" "$DOMAIN2_FIDDY_ADDRESS" "$domain2TokenMessengerAddress" "$attestor"

cd $1
ls $1
echo $2
export MESSAGE_TRANSMITTER_DEPLOYER_KEY=$3
export TOKEN_MESSENGER_DEPLOYER_KEY=$3
export TOKEN_MINTER_DEPLOYER_KEY=$3
export TOKEN_CONTROLLER_DEPLOYER_KEY=$3
export ATTESTER_ADDRESS=$7
export USDC_CONTRACT_ADDRESS=$4
export TOKEN_CONTROLLER_ADDRESS=$2
export BURN_LIMIT_PER_MESSAGE=1000
export REMOTE_USDC_CONTRACT_ADDRESS=$5
export DOMAIN=1
export REMOTE_DOMAIN=2
export MESSAGE_TRANSMITTER_PAUSER_ADDRESS=$2
export TOKEN_MINTER_PAUSER_ADDRESS=$2
export MESSAGE_TRANSMITTER_RESCUER_ADDRESS=$2
export TOKEN_MESSENGER_RESCUER_ADDRESS=$2
export TOKEN_MINTER_RESCUER_ADDRESS=$2
export REMOTE_TOKEN_MESSENGER_ADDRESS=$6


make simulate RPC_URL=http://localhost:8545 SENDER=$2

In [ ]:
%%bash -s "$cctp_home" "$domain_1_acct" "$domain_1_key" "$DOMAIN1_FIDDY_ADDRESS" "$DOMAIN2_FIDDY_ADDRESS" "$domain2TokenMessengerAddress" "$attestor"

cd $1
ls $1
echo $2
export MESSAGE_TRANSMITTER_DEPLOYER_KEY=$3
export TOKEN_MESSENGER_DEPLOYER_KEY=$3
export TOKEN_MINTER_DEPLOYER_KEY=$3
export TOKEN_CONTROLLER_DEPLOYER_KEY=$3
export ATTESTER_ADDRESS=$7
export USDC_CONTRACT_ADDRESS=$4
export TOKEN_CONTROLLER_ADDRESS=$2
export BURN_LIMIT_PER_MESSAGE=1000
export REMOTE_USDC_CONTRACT_ADDRESS=$5
export DOMAIN=1
export REMOTE_DOMAIN=2
export MESSAGE_TRANSMITTER_PAUSER_ADDRESS=$2
export TOKEN_MINTER_PAUSER_ADDRESS=$2
export MESSAGE_TRANSMITTER_RESCUER_ADDRESS=$2
export TOKEN_MESSENGER_RESCUER_ADDRESS=$2
export TOKEN_MINTER_RESCUER_ADDRESS=$2
export REMOTE_TOKEN_MESSENGER_ADDRESS=$6


make deploy RPC_URL=http://localhost:8545 SENDER=$2


In [ ]:
#export CCTPCTX=$HOME/code/evm-cctp-contracts/broadcast/deploy.s.sol/31337/run-latest.json
cctp_ctx = home[0] + '/code/evm-cctp-contracts/broadcast/deploy.s.sol/31337/run-latest.json'
print(cctp_ctx)

In [ ]:
%%bash -s "$exec_home" "$cctp_ctx" --out CCTP_AGGREGATE
cd $1
export CCTPCTX=$2
ls
node script/cctp-details.js

In [ ]:
CCTP_AGGREGATE = CCTP_AGGREGATE.strip()
parts = CCTP_AGGREGATE.split(':')
print(parts)

In [ ]:
domain1MessageTransmitterAddr = parts[0]
domain1TokenMessengerAddr = parts[1]
domain1TokenMinterAddr = parts[2]

In [ ]:
print(f'domain1MessageTransmitterAddr {domain1MessageTransmitterAddr}')
print(f'domain1TokenMessengerAddr     {domain1TokenMessengerAddr}')
print(f'domain1TokenMinterAddr        {domain1TokenMinterAddr}')

## CCTP Setup - Domain 2

In [ ]:
%%bash -s "$cctp_home" "$domain_2_acct" "$domain_2_key" "$DOMAIN2_FIDDY_ADDRESS" "$DOMAIN1_FIDDY_ADDRESS" "$domain1TokenMessengerAddr" "$attestor" "$PG_URL"

cd $1
ls $1
echo $2
export MESSAGE_TRANSMITTER_DEPLOYER_KEY=$3
export TOKEN_MESSENGER_DEPLOYER_KEY=$3
export TOKEN_MINTER_DEPLOYER_KEY=$3
export TOKEN_CONTROLLER_DEPLOYER_KEY=$3
export ATTESTER_ADDRESS=$7
export USDC_CONTRACT_ADDRESS=$4
export TOKEN_CONTROLLER_ADDRESS=$2
export BURN_LIMIT_PER_MESSAGE=1000
export REMOTE_USDC_CONTRACT_ADDRESS=$5
export DOMAIN=2
export REMOTE_DOMAIN=1
export MESSAGE_TRANSMITTER_PAUSER_ADDRESS=$2
export TOKEN_MINTER_PAUSER_ADDRESS=$2
export MESSAGE_TRANSMITTER_RESCUER_ADDRESS=$2
export TOKEN_MESSENGER_RESCUER_ADDRESS=$2
export TOKEN_MINTER_RESCUER_ADDRESS=$2
export REMOTE_TOKEN_MESSENGER_ADDRESS=$6


make simulate RPC_URL=$8 SENDER=$2

In [ ]:
%%bash -s "$cctp_home" "$domain_2_acct" "$domain_2_key" "$DOMAIN2_FIDDY_ADDRESS" "$DOMAIN1_FIDDY_ADDRESS" "$domain1TokenMessengerAddr" "$attestor" "$PG_URL"

cd $1
ls $1
echo $2
export MESSAGE_TRANSMITTER_DEPLOYER_KEY=$3
export TOKEN_MESSENGER_DEPLOYER_KEY=$3
export TOKEN_MINTER_DEPLOYER_KEY=$3
export TOKEN_CONTROLLER_DEPLOYER_KEY=$3
export ATTESTER_ADDRESS=$7
export USDC_CONTRACT_ADDRESS=$4
export TOKEN_CONTROLLER_ADDRESS=$2
export BURN_LIMIT_PER_MESSAGE=1000
export REMOTE_USDC_CONTRACT_ADDRESS=$5
export DOMAIN=2
export REMOTE_DOMAIN=1
export MESSAGE_TRANSMITTER_PAUSER_ADDRESS=$2
export TOKEN_MINTER_PAUSER_ADDRESS=$2
export MESSAGE_TRANSMITTER_RESCUER_ADDRESS=$2
export TOKEN_MESSENGER_RESCUER_ADDRESS=$2
export TOKEN_MINTER_RESCUER_ADDRESS=$2
export REMOTE_TOKEN_MESSENGER_ADDRESS=$6


make deploy RPC_URL=$8 SENDER=$2

In [ ]:
cctp_ctx = home[0] + '/code/evm-cctp-contracts/broadcast/deploy.s.sol/80001/run-latest.json'
print(cctp_ctx)

In [ ]:
%%bash -s "$exec_home" "$cctp_ctx" --out CCTP_AGGREGATE
cd $1
export CCTPCTX=$2
ls
node script/cctp-details.js

In [ ]:
CCTP_AGGREGATE = CCTP_AGGREGATE.strip()
parts = CCTP_AGGREGATE.split(':')
print(parts)

In [ ]:
domain2MessageTransmitterAddr = parts[0]
domain2TokenMessengerAddr = parts[1]
domain2TokenMinterAddr = parts[2]

In [ ]:
print(f'domain2MessageTransmitterAddr { domain2MessageTransmitterAddr }')
print(f'domain2TokenMessengerAddr     { domain2TokenMessengerAddr }')
print(f'domain2TokenMinterAddr        { domain2TokenMinterAddr }')

## Fix Token Messenger if Needed

***Note - looks like the first contract deployed is Message, which means it won't match the precompute - need to adjust logic***

In [ ]:
# Did the precompute match the deployment address?
print(domain2TokenMessengerAddr)
print(domain2TokenMessengerAddress)
domain2TokenMessengerAddr == domain2TokenMessengerAddress

In [ ]:
domain2TokenMessAddrBytes32 = bytes.fromhex(domain2TokenMessengerAddr[2:])
domain2TokenMessAddrBytes32 =  b'\0'*12 + domain2TokenMessAddrBytes32
print(domain2TokenMessAddrBytes32.hex())

In [ ]:
ethTokenMessengerAbiFile = home[0] + '/code/evm-cctp-contracts/out/TokenMessenger.sol/TokenMessenger.json'
print(ethTokenMessengerAbiFile)

In [ ]:
tokenMessengerAbiFile = home[0] + '/code/evm-cctp-contracts/out/TokenMessenger.sol/TokenMessenger.json'
print(tokenMessengerAbiFile)

In [ ]:
import json

with open(tokenMessengerAbiFile) as f:
    tm_json = json.load(f)

tokenMessengerAbi = tm_json['abi']

In [ ]:
print(tokenMessengerAbi)

In [ ]:
from web3 import Web3
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
domain1TokenMessenger = w3.eth.contract(address=domain1TokenMessengerAddr, abi=tokenMessengerAbi)

In [ ]:
domain1TokenMessenger.functions.remoteTokenMessengers(2).call().hex()

In [ ]:
tx = domain1TokenMessenger.functions.removeRemoteTokenMessenger(2).build_transaction({
    "from":domain_1_acct,
    "nonce": w3.eth.get_transaction_count(domain_1_acct),
})

In [ ]:
signed_tx = w3.eth.account.sign_transaction(tx, domain_1_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(tx_receipt)
print(f"Tx successful with hash: { tx_receipt.transactionHash.hex() }")

In [ ]:
tx = domain1TokenMessenger.functions.addRemoteTokenMessenger(2, domain2TokenMessAddrBytes32).build_transaction({
    "from":ACCT0,
    "nonce": w3.eth.get_transaction_count(domain_1_acct),
})

In [ ]:
signed_tx = w3.eth.account.sign_transaction(tx, domain_1_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(tx_receipt)
print(f"Tx successful with hash: { tx_receipt.transactionHash.hex() }")

In [ ]:
domain1TokenMessenger.functions.remoteTokenMessengers(2).call().hex()

## Working with Fiddy and CCTP on Domain 1

In [ ]:
fiddyAbiFile = home[0] + '/code/cctp-components/out/Token.sol/FiddyCent.json'

In [ ]:
import json

with open(fiddyAbiFile) as f:
    fiddy_json = json.load(f)

fiddyAbi = fiddy_json['abi']


In [ ]:
domain1Fiddy = w3.eth.contract(address=DOMAIN1_FIDDY_ADDRESS, abi=fiddyAbi)

In [ ]:
# Initial balance
domain1Fiddy.functions.balanceOf(ACCT1).call()

In [ ]:
# Transfer some fiddy coin to acct1
tx = domain1Fiddy.functions.transfer(ACCT1, 100).build_transaction(
    {
        "from":ACCT0,
        "nonce": w3.eth.get_transaction_count(ACCT0),
    }
)

In [ ]:
signed_tx = w3.eth.account.sign_transaction(tx, ACCT0KEY)

In [ ]:
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

print(f"Tx successful with hash: { tx_receipt.transactionHash.hex() }")

In [ ]:
domain1Fiddy.functions.balanceOf(ACCT1).call()

In [ ]:
# Check the allowance for ACCT1 on the messenger contract
domain1Fiddy.functions.allowance(ACCT1, domain1TokenMessengerAddr).call()

In [ ]:
# Set an allowance
tx = domain1Fiddy.functions.approve(domain1TokenMessengerAddr, 100).build_transaction(
    {
        "from":ACCT1,
        "nonce": w3.eth.get_transaction_count(ACCT1),
    }
)

In [ ]:
signed_tx = w3.eth.account.sign_transaction(tx, ACCT1KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

print(f"Tx successful with hash: { tx_receipt.transactionHash.hex() }")
print(f"New allowance: { domain1Fiddy.functions.allowance(ACCT1, domain1TokenMessengerAddr).call() }")

## Deposit for Burn - Domain 1 Side

In [ ]:
domain1TokenMessenger.functions.remoteTokenMessengers(2).call().hex()

In [ ]:
# Bytes 32 encoding of recipient
domain2MintReceiptientBytes32 = bytes.fromhex(domain_2_mint_recipient[2:])
domain2MintReceiptientBytes32 =  b'\0'*12 + domain2MintReceiptientBytes32
print(domain2MintReceiptientBytes32.hex())

In [ ]:
# Deposit some coinage for burn - args are amount, destination domain, mint recipient, contract that burns the deposit
tx = domain1TokenMessenger.functions.depositForBurn(1, 2, domain2MintReceiptientBytes32, DOMAIN1_FIDDY_ADDRESS).build_transaction(
     {
        "from":ACCT1,
        "nonce": w3.eth.get_transaction_count(ACCT1),
    }
)

In [ ]:
signed_tx = w3.eth.account.sign_transaction(tx, ACCT1KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(tx_receipt)
print(f"Tx successful with hash: { tx_receipt.transactionHash.hex() }")

In [ ]:
domain1Fiddy.functions.balanceOf(ACCT1).call()

In [ ]:
domain1Fiddy.functions.totalSupply().call()

## Message Transmitter Events

In [ ]:
msgTransmitterAbiFile = home[0] + '/code/evm-cctp-contracts/out/MessageTransmitter.sol/MessageTransmitter.json'
print(msgTransmitterAbiFile)

In [ ]:
with open(msgTransmitterAbiFile) as f:
    mt_json = json.load(f)

msgTransmitterAbi = mt_json['abi']

In [ ]:
print(msgTransmitterAbi)

In [ ]:
domain1MessageTransmitter = w3.eth.contract(address=domain1MessageTransmitterAddr, abi=msgTransmitterAbi)

In [ ]:
messageSentEvents = domain1MessageTransmitter.events.MessageSent.getLogs(fromBlock=1)
print(messageSentEvents)

In [ ]:
len(messageSentEvents)

In [ ]:
lastMessageSentEvent = messageSentEvents[len(messageSentEvents) - 1]

In [ ]:
lastMessageSentEvent

In [ ]:
messageSentBytes = lastMessageSentEvent.args.message
print(messageSentBytes.hex())

## Attestation



In [ ]:
mbHash = Web3.keccak(messageSentBytes)
signedHash = w3.eth.account.signHash(mbHash, private_key=attestor_key)

## Redeem Using Attested Message

In [ ]:
# To redeem on polygon, we call ReceiveMessage on message transmitter
domain2MessageTransmitter = mbw3.eth.contract(address=domain2MessageTransmitterAddr, abi=msgTransmitterAbi)

In [ ]:
# Check config
print(f'domain 2 transmitter local domain {domain2MessageTransmitter.functions.localDomain().call()}')
print(f'domain 2 attestor enabled? {domain2MessageTransmitter.functions.getEnabledAttester(0).call() == attestor}')

In [ ]:
messageSentBytes.hex()

In [ ]:
domain2TokenMessenger = mbw3.eth.contract(address=domain2TokenMessengerAddr, abi=tokenMessengerAbi)

In [ ]:
domain2TokenMessenger.functions.localMinter().call() == domain2TokenMinterAddr

In [ ]:

# Should fail - not a minter
tx = domain2MessageTransmitter.functions.receiveMessage(messageSentBytes,signedHash.signature).build_transaction({
    "from":domain_2_acct,
    "nonce": w3.eth.get_transaction_count(domain_2_acct),
})

In [ ]:
######

In [ ]:
domain2Fiddy = mbw3.eth.contract(address=DOMAIN2_FIDDY_ADDRESS, abi=fiddyAbi)

In [ ]:
domain2Fiddy.functions.MINTER_ROLE().call()

In [ ]:
tx = domain2Fiddy.functions.grantRole(domain2Fiddy.functions.MINTER_ROLE().call(),domain2TokenMinterAddr).build_transaction({
    "from":domain_2_acct,
    "nonce": mbw3.eth.get_transaction_count(domain_2_acct),
})


In [ ]:
signed_tx = mbw3.eth.account.sign_transaction(tx, domain_2_key)
tx_hash = mbw3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = mbw3.eth.wait_for_transaction_receipt(tx_hash)

print(f"Tx successful with hash: { tx_receipt.transactionHash.hex() }")


In [ ]:
domain2Fiddy.functions.hasRole(domain2Fiddy.functions.MINTER_ROLE().call(),domain2TokenMinterAddr).call()

In [ ]:
domain2TokenMessenger.functions.localMinter().call()

In [ ]:
# Should not throw a role-based exception
tx = domain2MessageTransmitter.functions.receiveMessage(messageSentBytes,signedHash.signature).build_transaction({
    "from":domain_2_acct,
    "nonce": mbw3.eth.get_transaction_count(domain_2_acct),
})

In [ ]:
signed_tx = mbw3.eth.account.sign_transaction(tx, domain_2_key)
tx_hash = mbw3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = mbw3.eth.wait_for_transaction_receipt(tx_hash)

print(f"Tx successful with hash: { tx_receipt.transactionHash.hex() }")

In [ ]:
domain2Fiddy.events.Transfer.getLogs(fromBlock=1)


In [ ]:
domain2Fiddy.functions.balanceOf(domain_2_mint_recipient).call()